In [ ]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
import schedule
import time
from datetime import datetime, timedelta
import os

# Setup the Open-Meteo API client
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

def fetch_weather_data():
    try:
        print("========================================================" )
        print(f" [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] start in extract data")
        
        yesterday = (datetime.now() - timedelta(days=1)).date()
        
        print(f"will be extract data: {yesterday} ")
        print("==========================================================" )
        
        # API parameters
        url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
        params = {
            "latitude": 52.52,
            "longitude": 13.41,
            "start_date": str(yesterday),  
            "end_date": str(yesterday),   
            "hourly": [
                "temperature_2m", "wind_speed_10m", "rain", 
                "showers", "snowfall", "cloud_cover_mid", 
                "cloud_cover_high", "cloud_cover_low", 
                "cloud_cover", "is_day", "sunshine_duration"
            ],
            "models": "kma_seamless",
        }
        
        # Fetch data from API
        responses = openmeteo.weather_api(url, params=params)
        response = responses[0]
        
        print(f" Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
        print(f" Elevation: {response.Elevation()} m asl")
        print(f" Timezone: GMT+{response.UtcOffsetSeconds()//3600}\n")
        
        # Process hourly data
        hourly = response.Hourly()
        
        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            )
        }
        
        # Extract variables
        variables = [
            "temperature_2m", "wind_speed_10m", 
            "rain", "showers", "snowfall", "cloud_cover_mid",
            "cloud_cover_high", "cloud_cover_low",
            "cloud_cover", "is_day", "sunshine_duration"
        ]
        
        for i, var_name in enumerate(variables):
            hourly_data[var_name] = hourly.Variables(i).ValuesAsNumpy()
        
        # Create DataFrame
        df = pd.DataFrame(data=hourly_data)
        
    
        if len(df) == 0:
            print("not found data for select date")
            print("=============================================="  "\n")
            return None
        
        print(f" reciveed data: {len(df)} \n")
        
        # Save to CSV with timestamp
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f'weather_data_{yesterday.strftime("%Y%m%d")}.csv'
        df.to_csv(filename, index=False)
        
        # Append to main file
        if os.path.exists('weather_data_all.csv'):
            existing_df = pd.read_csv('weather_data_all.csv')
            
            # check dialy data
            existing_df['date'] = pd.to_datetime(existing_df['date'])
            yesterday_data = existing_df[existing_df['date'].dt.date == yesterday]
            
            if len(yesterday_data) > 0:
                # delete old data
                existing_df = existing_df[existing_df['date'].dt.date != yesterday]
            
            # add new data
            combined_df = pd.concat([existing_df, df], ignore_index=True)
            combined_df['date'] = pd.to_datetime(combined_df['date'])
            combined_df = combined_df.sort_values('date')
            combined_df.to_csv('weather_data_all.csv', index=False)
            
            print(f" update in: weather_data_all.csv")
            print(f"total data: {len(combined_df)}")
        
        print(f"\n [{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}] mission done")
        print("===================================================================" "\n")
        
        return df
        
    except Exception as e:
        print(f"error: {e}")
        import traceback
        traceback.print_exc()
        print("======================================================"  "\n")
        return None

print("start in autometion for weather")
print("the data will be extract daliy in 12:06 AM")
print("will extract yesterday data daynamic\n")

schedule.every().day.at("12:06").do(fetch_weather_data)

fetch_weather_data()
print(" ctrl+c  to stop the program\n")

while True:
    schedule.run_pending()
    time.sleep(60)  # Check every minute


start in autometion for weather
the data will be extract daliy in 12:06 AM
will extract yesterday data daynamic

 [2026-02-15 01:42:04] start in extract data
will be extract data: 2026-02-14 
 Coordinates: 52.546875°N 13.4296875°E
 Elevation: 38.0 m asl
 Timezone: GMT+0

 reciveed data: 24 

 update in: weather_data_all.csv
total data: 36144

 [2026-02-15 01:42:05] mission done

 ctrl+c  to stop the program

